In [1]:
# FIXME refactor the rest of code and remove this line in the future
from __future__ import print_function, division

import itertools
import argparse
import numpy as np
import pandas as pd
import sklearn
import random
import sklearn.metrics
import time
import os
import sys
import json
import gc
import json
import hashlib
import tempfile
import keras.optimizers
import pickle
import datetime
import cv2
import math
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Reshape
import ast
import h5py
from keras.utils import np_utils
from itertools import tee
# import keras.backend as K
import tensorflow as tf
# from tensorflow import Keras as K

from copy import deepcopy
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from IPython.display import display, clear_output
import PIL.Image
#from cStringIO import StringIO
import IPython.display
import matplotlib.pyplot as plt
from util import get_csv_samples_many

%run exp_util.py

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.25
# keras.backend.set_session(tf.Session(config=config))

# import numpy as np
# import pandas as pd
# import random
# import pickle
# import cv2
# from copy import deepcopy
# import time
# import keras.optimizers
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Sequential, load_model
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Convolution2D, MaxPooling2D, Reshape
# from keras.utils import np_utils
# import keras.backend as K
# import gc, os, itertools
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.framework.graph_util import convert_variables_to_constants
# from util import get_csv_samples_many

# import matplotlib.pyplot as plt


# %run util.py



# FIXME for debuging
INIT_FLAG = False
VARIABLE_STORAGE = None

Using TensorFlow backend.


In [2]:
fgbg_dict = {}

def gen_model(resol, model_param, pre_model=None, model_type='AlexNet'):
    if model_type == 'AlexNet':
        param = ((resol[0], resol[1], 3), 2, model_param[0], model_param[1], model_param[2])
        model = generate_conv_net_base(*param, lr_mult=0.001)
    elif model_type == 'MobileNet':
        model = tf.keras.applications.mobilenet.MobileNet(
            input_shape=(resol[0], resol[1], 3),
            alpha=0.25, depth_multiplier=1, dropout=1e-3,
            include_top=True, weights=None, classes=2)
        model.compile(loss=get_loss(),
                  optimizer=get_optimizer(lr_mult=0.001),
                  metrics=computed_metrics)
    else:
        print ('No such model supported: ' + model_type)
    if pre_model != None:
        model.set_weights(pre_model.get_weights())
    return model

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [4]:
def str2ilst(s):
    return [int(x) for x in s.split(',')]

def add_json_file(key, val, fname):
    """
    Retrive data from given JSON file and update / insert an entry based on the given key
    """
    res = {}
    if os.path.exists(fname):
        with open(fname) as f:
            res = json.load(f)
    res[key] = val
    with open(fname, 'w+') as out_f:
        json.dump(res, out_f)
        
def get_score_fname(cur_cfg):
    """
    Take the value of cur_cfg and encode it by utf-8 and hash it.
    """
    scores_dir = os.path.join(CUR_RES_ROOT_PATH, 'scores')
    if not os.path.exists(scores_dir):
        os.mkdir(scores_dir)
    cfg_str = json.dumps(cur_cfg)
    cfg_md5 = hashlib.md5(cfg_str.encode("utf-8")).hexdigest()
    cfg_fname = os.path.join(scores_dir, cfg_md5)
    return cfg_fname

In [5]:
# Place for constants

FRAMES_PER_SEC = 10 # frame rate we decode the video
FULL_RESOL = (720, 1280) # default 720p

RES_ROOT_PATH="/media/YOLO-RES-720P/exp"
JPG_ROOT_PATH="/media/YOLO-RES-720P/jpg"
CSV_ROOT_PATH="/media/YOLO-RES-720P/out" 

In [6]:
# Refactor
# * epoch should be passed in from outter function
# * wnd_num? what is that?
# * remove K (kears), use TF.kears to replace it
def add_json_file(key, val, fname):
    res = {}
    if os.path.exists(fname):
        with open(fname) as f:
            res = json.load(f)
    res[key] = val
    with open(fname, 'w+') as out_f:
        json.dump(res, out_f)

def run_models(X_train, Y_train, X_test, Y_test, video_cfg, cur_cfg):
    nb_epoch=10
    wnd_num = cur_cfg['input_cfg'][2]
    out_file = os.path.join(CUR_RES_ROOT_PATH, cur_cfg['video'] + '.json')
    resol = cur_cfg['resol']
#     X_train_total = np.concatenate(X_train_wnds, axis=0)
#     Y_train_total = np.concatenate(Y_train_wnds, axis=0)
#     X_test_total = np.concatenate(X_test_wnds, axis=0)
#     Y_test_total = np.concatenate(Y_test_wnds, axis=0)
    
    mean = np.mean(X_train, axis=0)
    save_mean(cur_cfg, mean)
    X_train = X_train - mean
    X_test = X_test - mean
    
    max_conv_num = int(math.log(min(resol), 2))
    cfgs = []
    for conv_num in range(max_conv_num):
        for cfg in video_cfg['model']:
            cfgs.append(cfg + ',' + str(conv_num + 1))
    
    # FIXME stop here, check parameters
    #print(f'cfgs: {cfgs}')
    #return
    # FIXME remember to remove the above
    
    for model_cfg in cfgs:
        print ('running cfg: ' + model_cfg)
        cur_cfg['model'] = model_cfg
        param = str2ilst(model_cfg)
        #K.clear_session()
        # FIXME compatible
        tf.compat.v2.keras.backend.clear_session()
        
        cur_cfg['generic'] = True
        if already_saved(json.dumps(cur_cfg), out_file):
            print ('skip...')
            continue
        generic_model = gen_model(resol, param)
        
        # FIXME train model indicator
        train_start = time.time()
        
        print("----------------------Training---------------------------------")
        
        train_model(generic_model, X_train, Y_train, batch_size=32,
                    nb_epoch=20, save_path=get_model_fname(cur_cfg))
        
        
        
        # FIXME train model indicator
        train_done = time.time()
        print(f'time diff {train_done - train_start}')
        print("----------------------Training done---------------------------------")
        
        
        scores = test_model(generic_model, X_test, Y_test)
        
        # FIXME debuging
        global INIT_FLAG
        global VARIABLE_STORAGE
        if (not INIT_FLAG):
            VARIABLE_STORAGE = scores
            INIT_FLAG = True
            print('VARIABLE_STORAGE')
            print(VARIABLE_STORAGE)
            print(f'out_file: {out_file}')
            print(f'get_score_fname(cur_cfg): {get_score_fname(cur_cfg)}')
            raise MemoryError("STOP HERE, debug")

        add_json_file(json.dumps(cur_cfg), str(datetime.datetime.now()), out_file)

        with open(get_score_fname(cur_cfg), 'wb+') as f:
            np.save(f, scores)
        
        cur_cfg['generic'] = False
        total_scores = []
        wnd_size_train = X_train.shape[0] // wnd_num
        wnd_size_test = X_test.shape[0] // wnd_num
        for i in range(wnd_num):
            start_train = i * wnd_size_train
            end_train = (i + 1) * wnd_size_train
            start_test = i * wnd_size_test
            end_test = (i + 1) * wnd_size_test
            model = gen_model(resol, param, pre_model=generic_model)
            train_model(model, X_train[start_train:end_train], Y_train[start_train:end_train],
                        batch_size=32, nb_epoch=10, save_path=get_model_fname(cur_cfg) + '_' + str(i))
            res = test_model(model, X_test[start_test:end_test], Y_test[start_test:end_test])
            total_scores.append(res)
        add_json_file(json.dumps(cur_cfg), str(datetime.datetime.now()), out_file)
        total_scores = np.concatenate(total_scores, axis=0)
        with open(get_score_fname(cur_cfg), 'wb+') as f:
            np.save(f, total_scores)
 

In [7]:
def resize_images(frames, resol, dtype='float32'):
    new_frames = np.zeros(tuple([frames.shape[0]] + list(resol) + [3]), dtype=dtype)
    for i in range(frames.shape[0]):
        one_frame = cv2.resize(frames[i,:,:,:], (resol[1], resol[0]), interpolation=cv2.INTER_NEAREST)
        new_frames[i, :] = one_frame
    return new_frames

def cfg2mean(cfg):
    tag = ','.join([str(cfg['resol'][0]), str(cfg['resol'][1]), cfg['input_cfg'][-1]])
    return tag

def save_mean(cfg, mean):
    cfg_tag = cfg2mean(cfg)
    mean_dir = os.path.join(RES_ROOT_PATH, cfg['video'], 'mean')
    if not os.path.exists(mean_dir):
        os.mkdir(mean_dir)
    mean_fname = os.path.join(mean_dir, cfg_tag)
    np.save(mean_fname, mean)

In [8]:
def read_test_data_ids(cur_cfg, wnd_id, candidate_ids):
    video = cur_cfg['video']
    obj = cur_cfg['OBJECT']
    testids_dir = os.path.join(CUR_RES_ROOT_PATH, 'testids')
    if not os.path.exists(testids_dir):
        os.mkdir(testids_dir)
    testid_fname = os.path.join(testids_dir, video + '-' + obj + '-' + str(wnd_id))
    if not os.path.exists(testid_fname):
        print('Create test id file...')
        with open(testid_fname, 'wb') as f:
            pickle.dump(candidate_ids, f)
    with open(testid_fname, 'rb') as f:
        return pickle.load(f)

In [9]:
def get_big_box(matches):
    min_x = min(m[0] for m in matches)
    min_y = min(m[1] for m in matches)
    max_x = max(m[2] for m in matches)
    max_y = max(m[3] for m in matches)
    return (min_x, min_y, max_x, max_y)

def contain_rect(br, sr):
    return br[0] <= sr[0] and br[1] <= sr[1] and br[2] >= sr[2] and br[3] >= sr[3]

def convert(crop):
    return (crop[1], crop[0], crop[1] + crop[3], crop[0] + crop[2])

def get_frames_from_images(frameset, img_dirs, img_nums, fgmask_dict=None,
                           resol=(-1, -1), crop=(-1, -1, -1, -1), dtype='float32'):
    def get_image_path(idx):
        for i in range(len(img_dirs)):
            if sum(img_nums[:i + 1]) > idx:
                return os.path.join(img_dirs[i], str(idx - sum(img_nums[:i]) + 1).zfill(7) + '.jpg')
    assert len(img_dirs) == len(img_nums), 'image subdir numbers not consistent!'
#     print ('reading images... %d from %s...') % (len(frameset), img_dirs[0])
    if resol[0] > 0:
        frames = np.zeros(tuple([len(frameset)] + list(resol) + [3]), dtype=dtype )
    else:
        test_frame = cv2.imread(os.path.join(img_dirs[0], '0000001.jpg'))
        frames = np.zeros(tuple([len(frameset)] + list(test_frame.shape)), dtype=dtype )
    for i in range(len(frameset)):
        img_path = get_image_path(frameset[i])
        frame = cv2.imread(img_path)
        tmp_crop = deepcopy(crop)
        if fgmask_dict is not None:
            tmp_idx = frameset[i]
            while tmp_idx not in fgmask_dict:
                tmp_idx -= 1
            boxes = fgmask_dict[tmp_idx]
            boxes = [b for b in boxes if b[2] > 40 and b[3] > 40]
            boxes = [convert(b) for b in boxes]
            if tmp_crop[0] > 0:
                boxes = [b for b in boxes if contain_rect(tmp_crop, b)]
            if len(boxes) > 0:
                tmp_crop = get_big_box(boxes)
        if tmp_crop[0] > 0:
            frame = frame[tmp_crop[0]:tmp_crop[2],tmp_crop[1]:tmp_crop[3],:]
        if resol[0] > 0:
            frame = cv2.resize(frame, (resol[1], resol[0]), interpolation=cv2.INTER_NEAREST)
        frames[i, :] = frame

    if dtype == 'float32':
        frames /= 255.0

    return frames


In [10]:
def get_labels(obj_ids, frameset):
    ret = [t in obj_ids for t in frameset]
    print('reading label sum: %d, pos %d'%(len(ret), sum(ret)))
    return np_utils.to_categorical(ret, 2)

def already_saved(key, fname):
    if os.path.exists(fname):
        with open(fname) as f:
            res = json.load(f)
            if key in res: return True
    return False

In [11]:


def read_input(obj_ids, input_cfg, wnd_size_lst, img_sub_dirs,
               img_sub_nums, cur_cfg, max_resol=(100, 100)):
    
    # FIXME this function take long time to run... Need to optimize this function
    print('reading input for config:', input_cfg)
    train_num = input_cfg[0]
    test_num = input_cfg[1]
    wnd_num = input_cfg[2]
    train_pos_ratio = input_cfg[3]
    crop = str2ilst(input_cfg[4])
    train_num_wnd = train_num // wnd_num
    test_num_wnd = test_num // wnd_num
    train_ids = []
    test_ids = []
    
    for i in range(wnd_num):
        ids_wnd = range(sum(wnd_size_lst[:i]), sum(wnd_size_lst[:i+1]), 1)
        test_ids_candidate = random.sample(ids_wnd, test_num_wnd)
        test_ids_wnd = read_test_data_ids(cur_cfg, i, test_ids_candidate)
        if len(test_ids_wnd) != test_num_wnd:
            print('test id number not matched!!!!!')
            sys.exit()
        
        temp_ids_wnd = set(ids_wnd) - set(test_ids_wnd)
        train_ids_pos_wnd_total = set(temp_ids_wnd) & set(obj_ids)
        real_pos_ratio = float(len(train_ids_pos_wnd_total)) / len(temp_ids_wnd)
        if train_pos_ratio < real_pos_ratio:
            train_ids_wnd = random.sample(temp_ids_wnd, train_num_wnd)
        else:
            pos_num_needed = int(train_num_wnd * train_pos_ratio)
            pos_num_needed = min(
                [
                    pos_num_needed,
                    # at least 30 frames
                    int(real_pos_ratio * train_num_wnd * 30),
                    len(train_ids_pos_wnd_total) // 2
                ])
            train_ids_pos_wnd = random.sample(
                train_ids_pos_wnd_total, pos_num_needed)
            train_ids_neg_wnd = random.sample(
                temp_ids_wnd - train_ids_pos_wnd_total,
                train_num_wnd - pos_num_needed)
            train_ids_wnd = train_ids_pos_wnd + train_ids_neg_wnd
            random.shuffle(train_ids_wnd)
        train_ids += train_ids_wnd
        test_ids += test_ids_wnd

    X_train = get_frames_from_images(
        train_ids, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop)
    Y_train = get_labels(obj_ids, train_ids)
    X_test = get_frames_from_images(
        test_ids, img_sub_dirs, img_sub_nums, resol=max_resol, crop=crop)
    Y_test = get_labels(obj_ids, test_ids)
    
    return X_train, Y_train, X_test, Y_test

In [12]:

def run_once(video_name, video_cfg):
    # iterate input formats
    video_clip_num = video_cfg['VIDEO_CLIP_NUM']
    SMOOTH_WINDOW = FRAMES_PER_SEC * video_cfg['SMOOTH_SEC'] # for smooth the yolo output
    input_cfgs = list(itertools.product(
        video_cfg['train_num'],
        video_cfg['test_num'],
        video_cfg['wnd_num'],
        video_cfg['train_pos_ratio'],
        video_cfg['crop']
    ))
    print(f"input_cfgs: {input_cfgs}")
    print()
    
    sub_videos = [video_name + '-' + str(p + 1) + '_10FPS' for p in range(video_clip_num)]
    print(f"sub_videos: {sub_videos}")
    print()
    
    img_sub_dirs = [os.path.join(JPG_ROOT_PATH, v) for v in sub_videos]
    print(f"img_sub_dirs: {img_sub_dirs}")
    print()
    
    img_sub_nums = [len(os.listdir(d)) for d in img_sub_dirs]
    print(f"img_sub_nums: {img_sub_nums}")
    print()
    
    total_frame_num = sum(img_sub_nums)
    
    csv_sub_files = [os.path.join(CSV_ROOT_PATH, d + '.csv') for d in sub_videos]
    print(f"csv_sub_files {csv_sub_files}")
    print()
    
    cur_cfg = {'video': video_name, 'smooth_window': SMOOTH_WINDOW}
    for OBJECT in video_cfg['OBJECT']:
        cur_cfg['OBJECT'] = OBJECT
        
        for input_cfg in input_cfgs:
            cur_cfg['input_cfg'] = input_cfg
            
            
            obj_ids, _ = get_csv_samples_many(
                csv_sub_files, img_sub_nums, OBJECT, total_frame_num,
                WINDOW=SMOOTH_WINDOW, crop=str2ilst(input_cfg[4]))

            # a list of integers indicate available sizes / resolution of vide frames
            resol_lst = [str2ilst(r) for r in video_cfg['resol']]

            # FIXME better way to to find this number
            max_resol = (max([r[0] for r in resol_lst]), max([r[1] for r in resol_lst]))
            
            X_train, Y_train, X_test, Y_test = read_input(
                obj_ids, input_cfg, img_sub_nums, img_sub_dirs,
                img_sub_nums, cur_cfg, max_resol=max_resol)
            
            for resol in resol_lst:
                cur_cfg['resol'] = resol
                if resol[0] != max_resol[0] or resol[1] != max_resol[1]:
                    resized_X_train = resize_images(X_train, resol, dtype='float32')
                    resized_X_test = resize_images(X_test, resol, dtype='float32')
                else:
                    resized_X_train = X_train
                    resized_X_test = X_test
                run_models(resized_X_train, Y_train, resized_X_test, Y_test, video_cfg, cur_cfg)
                resized_X_train = None
                resized_X_test = None
                gc.collect()
            
            X_train = None
            X_test = None
            Y_train = None
            Y_test = None

In [13]:
# automate experiments
# with open('exp_params.cfg') as f:
#     cfg = json.load(f)

cfg = {
    "venice-canal": {
        "VIDEO_CLIP_NUM": 8,
        "train_num": [
            24000
        ],
        "test_num": [
            16000
        ],
        "wnd_num": [
            8
        ],
        "OBJECT": [
            "person"
        ],
        "resol": [
            "25,25",
            "50,50",
            "100,100"
        ],
        "crop": [
            "420,780,720,1280"
        ],
        "train_pos_ratio": [
            -1
        ],
        "SMOOTH_SEC": 3,
        "model": [
            "16,8",
            "32,16",
            "64,32"
        ]
    }
}


# FIXME modify the place to store output
run_video = 'venice-canal'
CUR_RES_ROOT_PATH = os.path.join(RES_ROOT_PATH, run_video)

if not os.path.exists(CUR_RES_ROOT_PATH):
    
    raise Exception(f"{CUR_RES_ROOT_PATH} does not exit")

for video in cfg.keys():
    if video != run_video: 
        continue
    run_once(video, cfg[video])
    gc.collect()



input_cfgs: [(24000, 16000, 8, -1, '420,780,720,1280')]

sub_videos: ['venice-canal-1_10FPS', 'venice-canal-2_10FPS', 'venice-canal-3_10FPS', 'venice-canal-4_10FPS', 'venice-canal-5_10FPS', 'venice-canal-6_10FPS', 'venice-canal-7_10FPS', 'venice-canal-8_10FPS']

img_sub_dirs: ['/media/YOLO-RES-720P/jpg/venice-canal-1_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-2_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-3_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-4_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-5_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-6_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-7_10FPS', '/media/YOLO-RES-720P/jpg/venice-canal-8_10FPS']

img_sub_nums: [216233, 215774, 216102, 216012, 215863, 216052, 216003, 216150]

csv_sub_files ['/media/YOLO-RES-720P/out/venice-canal-1_10FPS.csv', '/media/YOLO-RES-720P/out/venice-canal-2_10FPS.csv', '/media/YOLO-RES-720P/out/venice-canal-3_10FPS.csv', '/media/YOLO-RES-720P/out/venice-canal-4_10FPS.csv', '/media/YOLO-RES-720P/

W1107 18:42:53.125545 140599609812800 training.py:706] The `nb_epoch` argument in `fit` has been renamed `epochs`.


----------------------Training---------------------------------
training samples: 3129/24000
Epoch 1/20


W1107 18:42:53.632467 140599609812800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:466: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


24000/24000 [==============================] - 5s 223us/sample - loss: 0.3540 - accuracy: 0.8682 - mean_squared_error: 0.1062
Epoch 2/20
24000/24000 [==============================] - 3s 110us/sample - loss: 0.2990 - accuracy: 0.8696 - mean_squared_error: 0.0905
Epoch 3/20
24000/24000 [==============================] - 3s 114us/sample - loss: 0.2824 - accuracy: 0.8725 - mean_squared_error: 0.0847
Epoch 4/20
24000/24000 [==============================] - 3s 106us/sample - loss: 0.2692 - accuracy: 0.8838 - mean_squared_error: 0.0794
Epoch 5/20
24000/24000 [==============================] - 3s 106us/sample - loss: 0.2582 - accuracy: 0.8875 - mean_squared_error: 0.0764
Epoch 6/20
24000/24000 [==============================] - 3s 112us/sample - loss: 0.2567 - accuracy: 0.8923 - mean_squared_error: 0.0754
Epoch 7/20
24000/24000 [==============================] - 3s 105us/sample - loss: 0.2499 - accuracy: 0.9011 - mean_squared_error: 0.0730
Epoch 8/20
24000/24000 [============================

MemoryError: STOP HERE, debug

## Training operator

1. Configuration (some parameters)
2. CSV file that contains information at which frame what objects were detected.
3. specify epoch

In [14]:
!ls /media/YOLO-RES-720P/exp/jackson/mean

100,100,-1,-1,-1,-1.npy     25,50,-1,-1,-1,-1.npy     50,50,-1,-1,-1,-1.npy
100,100,200,0,720,1280.npy  25,50,200,0,720,1280.npy  50,50,200,0,720,1280.npy


is equal [[ True  True]
 [ True  True]
 [ True  True]
 ...
 [ True  True]
 [ True  True]
 [ True  True]]
